In [1]:
import pyarrow.feather as feather
import sys
import pandas
from pandas.tseries.offsets import BDay, DateOffset
from datetime import date, datetime
import numpy
import sys, time, random

In [2]:
# preparing holdin periods and confidence levels list
HP = 1
HP_STEP = 1
HP_CNT = 2
CONF = 90.00
CONF_STEP =1.00
CONF_CNT = 3
LOOKBACK_PERIOD=1000

CLstart = CONF
CLlist = [CLstart]

while len(CLlist) < CONF_CNT:
    
    CL = CLstart + CONF_STEP
    CLlist.append(CL)
    CLstart = CL
    
print(CLlist)

[90.0, 91.0, 92.0]


In [3]:
HPstart = HP
HPlist = [HPstart]

while len(HPlist) < HP_CNT:
    
    HP = HPstart + HP_STEP
    HPlist.append(HP)
    HPstart = HP
    
print(HPlist)

[1, 2]


In [4]:
#load feather files
curves_df = feather.read_feather("./retrieved_curves.feather")
print(curves_df)

       rf_id   rf_dttm  rf_value  rf_node
0    ITA10YZ  20200702  12.10800      120
1    ITA10YZ  20200723   0.99043      120
2    ITA10YZ  20200727   0.99579      120
3    ITA10YZ  20200729  10.00100      120
4    ITA10YZ  20200730   0.97361      120
..       ...       ...       ...      ...
510   ITA9YZ  20200819   0.86550      108
511   ITA9YZ  20200820   0.86123      108
512   ITA9YZ  20200821   0.89252      108
513   ITA9YZ  20200823   0.89252      108
514   ITA9YZ  20200824   0.89331      108

[515 rows x 4 columns]


In [5]:
positions_df = feather.read_feather('./positions.feather')
print(positions_df)

      position_id  iter_id  account_id  instrument_id   qty       ctv  price
0            3200        0          33              0  6534  653400.0  100.0
1            3201        0          33              1  5430  570150.0  105.0
2            3202        0          33              2  9451  992355.0  105.0
3            3203        0          33              3  4184  435136.0  104.0
4            3204        0          33              4  7811  773289.0   99.0
...           ...      ...         ...            ...   ...       ...    ...
8595       214535        0          45             25  3865  386500.0  100.0
8596       214536        0          45             47  2150  225750.0  105.0
8597       214537        0          45             39  3491  366555.0  105.0
8598       214538        0          45             14  3853  396859.0  103.0
8599       214539        0          45             41  8169  816900.0  100.0

[8600 rows x 7 columns]


In [6]:
instrumentRf_df = feather.read_feather('./instrumentRf.feather')
print(instrumentRf_df)

     instrument_id          rf_id  bucket_flow
0                0         POR8YZ         22.0
1                1         EUR4YZ         51.0
2                1      EUROIS6MZ         73.0
3                2      ESGOV18YZ         20.0
4                2  EURESTOIS10YZ         84.0
..             ...            ...          ...
262             98         DEN1YZ         32.0
263             98         EUR3YZ         71.0
264             99        DEN10YZ         36.0
265             99         DEN9YZ         94.0
266             99     EUROIS25YZ         75.0

[267 rows x 3 columns]


# Scenario_unit

In [7]:
def createScenarios(hp_list, lookback_period, retrieved_curves):
	used_nodes = retrieved_curves['rf_node'].drop_duplicates().tolist()

	# scenarios construction
	scenarios = pandas.DataFrame()
	for holding_period in hp_list:
		for node_str in used_nodes:
			node = int(node_str)
			
			subframe = retrieved_curves.loc[retrieved_curves['rf_node'] == node_str].copy()
			subframe.sort_values(by=['rf_dttm'], ascending=False, inplace=True)
			
			shifted_prices = subframe['rf_value'][holding_period:].tolist()
			subframe = subframe[:-holding_period]
			
			subframe['SH_CLOSEPR'] = shifted_prices
			
			subframe['rf_value'] = subframe['rf_value'].astype(float)
			subframe['rf_node'] = subframe['rf_node'].astype(int)
			subframe['SH_CLOSEPR'] = subframe['SH_CLOSEPR'].astype(float)

			if node < 1:
				subframe['PRICE'] = 100 * numpy.exp(-subframe['rf_node'] * subframe['rf_value'] / 100)
				subframe['SH_PRICE'] = 100 * numpy.exp(-subframe['rf_node'] * subframe['SH_CLOSEPR'] / 100)
			else:
				subframe['PRICE'] = 100 / (1 + subframe['rf_value'] / 100) ** subframe['rf_node']
				subframe['SH_PRICE'] = 100 / (1 + subframe['SH_CLOSEPR'] / 100) ** subframe['rf_node']

			subframe['SN_VALUE'] = subframe['PRICE'] / subframe['SH_PRICE']
			subframe = subframe[:lookback_period]
			subframe['HP'] = [holding_period] * len(subframe.index)
			       
			scenarios = scenarios.append(subframe)

	scenarios.reset_index(inplace=True, drop=True)

	return scenarios

In [8]:
scenarios_df = createScenarios(HPlist,LOOKBACK_PERIOD,curves_df)
print(scenarios_df)

       rf_id   rf_dttm  rf_value  rf_node  SH_CLOSEPR      PRICE    SH_PRICE  \
0    ITA10YZ  20200824   0.96247      120     0.95901  31.681372   31.811929   
1    ITA10YZ  20200823   0.95901      120     0.95901  31.811929   31.811929   
2    ITA10YZ  20200821   0.95901      120     0.92875  31.811929   32.977115   
3    ITA10YZ  20200820   0.92875      120     0.92908  32.977115   32.964178   
4    ITA10YZ  20200819   0.92908      120     0.94104  32.964178   32.498777   
..       ...       ...       ...      ...         ...        ...         ...   
980   ITA9YZ  20200724   0.93721      108     0.97876  36.514049   34.926607   
981   ITA9YZ  20200723   0.92778      108    11.21100  36.884354    0.001038   
982   ITA9YZ  20200722   0.97876      108    11.21100  34.926607    0.001038   
983   ITA9YZ  20200719  11.21100      108     1.19100   0.001038   27.840442   
984   ITA9YZ  20200717  11.21100      108    -0.43002   0.001038  159.268780   

         SN_VALUE  HP  
0        0.9958

# Margin_unit

In [9]:
def portfolioAggregation(position,instrRf_df):
	position = position
	final_merge = pandas.DataFrame()

	for p in position['position_id'].tolist():
##		isin_map = mkv.getInstrumentRf(company,p)
		isin_frame = instrRf_df

		pos_subframe = position.loc[position['position_id'] == p]

		member_merge = pandas.merge(isin_frame, pos_subframe, on='instrument_id', how='inner')

		member_merge['ctv'] = member_merge['ctv'].astype(float)
		member_merge['bucket_flow'] = member_merge['bucket_flow'].astype(float)

		member_merge['FLOW_CTV'] = member_merge['bucket_flow'] / 100 * member_merge['ctv']

		member_merge = member_merge[['account_id', 'rf_id', 'FLOW_CTV']].copy()
		member_merge.sort_values(by=['account_id', 'rf_id'], ascending=True, inplace=True)
		member_merge.reset_index(inplace=True, drop=True)

		final_merge = final_merge.append(member_merge)

	final_merge = pandas.pivot_table(final_merge, index=['rf_id', 'account_id'], aggfunc=numpy.sum).reset_index()

	return final_merge

In [10]:
aggrAccountRf_df = portfolioAggregation(positions_df,instrumentRf_df)
print(aggrAccountRf_df)

       rf_id  account_id     FLOW_CTV
0    AUT10YZ          33   8013247.20
1    AUT10YZ          45   9985936.95
2    AUT20YZ          33   2393998.56
3    AUT20YZ          45   2968133.16
4     AUT3MZ          33  51354902.06
..       ...         ...          ...
369   SPA6YZ          45  34866011.20
370   SPA8YZ          33   7220141.12
371   SPA8YZ          45   6569008.00
372   SPA9YZ          33   7960840.24
373   SPA9YZ          45   6051972.20

[374 rows x 3 columns]


In [11]:
def expectedShortfall(scenario, position_rf, HPlist, CLlist):
	scenario_frame = scenario

	es_result = []
	hp_TOT, cl_TOT = [], []
	acc_id = []
    
	for hp in HPlist:
		scen_hp = scenario_frame.loc[scenario_frame['HP'] == hp]
		node_reval_frames = []
		for node in position_rf['rf_id'].tolist():
		    # loc and iloc methods are needed to extract rows from a data frame
		    # iloc needs an index; loc can also use boolean selection
		    flow_to_reval = position_rf.loc[position_rf['rf_id'] == node]['FLOW_CTV'].iloc[0]

		    sce_subframe = scen_hp.loc[scen_hp['rf_id'] == node].copy()
		    SUB = numpy.array(sce_subframe['SN_VALUE'].astype('float32'))
			#rivalutazione
		    sce_subframe['G/L_{}'.format(node)] = SUB * flow_to_reval - flow_to_reval
		    sce_subframe = sce_subframe[['rf_dttm', 'G/L_{}'.format(node)]].copy()
		    sce_subframe.reset_index(inplace=True, drop=True)
		   # print(sce_subframe.info())
		    node_reval_frames.append(sce_subframe)

		es_frame = node_reval_frames[0]

		for node_reval in node_reval_frames[:]:
			es_frame = pandas.merge(es_frame, node_reval, on=['rf_dttm'], how='inner')

		es_frame['PNL'] = es_frame.iloc[:, 1:].sum(axis=1)

		es_frame = es_frame[['rf_dttm', 'PNL']].copy()
		es_frame.sort_values(by=['PNL'], ascending=True, inplace=True)
		es_frame.reset_index(inplace=True, drop=True)
		
		for cl in CLlist:
			num_events_tail = max(int(round(len(es_frame.index) * (1 - cl), 0)), 1)
			es_frame = es_frame[:num_events_tail]

			es = es_frame['PNL'].mean()
			es_result.append(abs(es))

			cl_TOT.append(cl)
			hp_TOT.append(hp)
			acc_id.append(position_rf['account_id'].iloc[0])
        
	results_frame = pandas.DataFrame({'account_id': acc_id,
				      'EXPECTED_S': es_result,        
				      'HP': hp_TOT,
				      'CONF': cl_TOT})

	return results_frame

In [12]:
expShortfall_df = expectedShortfall(scenarios_df,aggrAccountRf_df,HPlist,CLlist)
print(expShortfall_df)

   account_id  EXPECTED_S  HP  CONF
0          33         NaN   1  90.0
1          33         NaN   1  91.0
2          33         NaN   1  92.0
3          33         NaN   2  90.0
4          33         NaN   2  91.0
5          33         NaN   2  92.0
